In [319]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import OneHotEncoder
import time

Use LOF and Isolation forest to find anomalies. At the end, I ended up using only anomalies flagged by Isolation forest for my further analysis

In [320]:
df_contract = pd.read_csv("contract_eng_consolidated.csv")
df_project = pd.read_csv('project_eng_consolidated_with_topics.csv')

C:\Users\v-krvibh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [185]:
def find_columns_with_null(df):
    null_columns = df.columns[df.isnull().any()]
    return null_columns 

def find_null_columns_value(df):
    null_columns = find_columns_with_null(df)
    return df[null_columns].isnull().sum()
    
def find_column_percent_null(df):
    null_columns = find_columns_with_null(df)
    null_columns_percent = df[null_columns].isnull().sum()/len(df)
    return null_columns_percent

In [209]:
print(df_project.columns)
print(df_project.shape)

Index(['project_type_name_eng', 'purchase_method_group_name_eng',
       'project_status_eng', 'sum_price_agree_cleaned', 'dept_name_group',
       'Topic_group'],
      dtype='object')
(397344, 6)


In [187]:
for i in df_project.columns:
    print(i, df_project[i].nunique())

project_type_name_eng 8
purchase_method_group_name_eng 11
province_eng 78
project_status_eng 2
district_eng 897
announce_date_eng 1495
transaction_date_eng 1660
project_money_cleaned 114553
price_build_cleaned 180204
sum_price_agree_cleaned 143179
dept_name_eng 17977
budget_year_cleaned 5
project_id 400988
project_name 375355
geom 53338
dept_name_group 7
Topic_group 20


In [207]:
df_project['purchase_method_group_name_eng'].unique()

array(['Bidding', 'E-bidding', 'Special', 'Hire consultants',
       'Special cases', 'Price examination', 'Agreed price', 'E-market',
       'selection', 'Specific'], dtype=object)

In [277]:
### drop category that have less than 0.1% of the total row count
df_project.drop(df_project[df_project['project_type_name_eng'] == 'Hire to design'].index, inplace=True)
df_project.drop(df_project[df_project['project_type_name_eng'] == 'Hire to design and supervise'].index, inplace=True)
df_project.drop(df_project[df_project['project_type_name_eng'] == 'Hire to supervise'].index, inplace=True)

df_project.drop(df_project[df_project['purchase_method_group_name_eng'] == 'Bidding'].index, inplace=True)
df_project.drop(df_project[df_project['purchase_method_group_name_eng'] == 'Agreed price'].index, inplace=True)
df_project.drop(df_project[df_project['purchase_method_group_name_eng'] == 'E-market'].index, inplace=True)
df_project.reset_index(drop=True,inplace = True)

In [278]:
### delete columns that have too many unique categorical values
detector_data = df_project.drop(columns=['province_eng','district_eng','announce_date_eng','transaction_date_eng','project_money_cleaned'])
detector_data.drop(columns=['geom','project_id','price_build_cleaned','dept_name_eng','project_name','budget_year_cleaned'],inplace = True)

In [279]:
detector_data.dtypes

project_type_name_eng             object
purchase_method_group_name_eng    object
project_status_eng                object
sum_price_agree_cleaned            int64
dept_name_group                   object
Topic_group                        int64
dtype: object

In [280]:
### standard scale numeric col
### turn string col to onehot

# Select numerical columns
#numerical_cols = [cname for cname in df_project.columns if 
#                df_project[cname].dtype in ['int64', 'float64']]

numerical_cols = ['sum_price_agree_cleaned']

In [321]:
categorical_cols = [cname for cname in detector_data.columns if
                    detector_data[cname].nunique() < 12 and 
                    detector_data[cname].dtype == "object"]
categorical_cols.append('Topic_group')

In [284]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) #handle_unknown='ignore': if test has that new feature, ignore it
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(detector_data[categorical_cols]))

In [285]:
OH_cols_train.index = df_project.index

In [286]:
OH_cols_train.columns = OH_encoder.get_feature_names()

In [287]:
OH_cols_train

,x0_Hire consultants,x0_Hire for construction,x0_Outsourcing / Contracting services,x0_buy,x0_rent,x1_E-bidding,x1_Hire consultants,x1_Price examination,x1_Special,x1_Special cases,...,x4_10,x4_11,x4_12,x4_13,x4_14,x4_15,x4_16,x4_17,x4_18,x4_19
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397339,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397340,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397341,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397342,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [288]:
X = pd.concat([detector_data[numerical_cols], OH_cols_train.reset_index(drop=True)], axis=1)

In [323]:
current = time.time()


outlier_detect = IsolationForest(n_estimators=100, max_samples=0.8, contamination=.001,max_features = 0.8, random_state = 4) 
# might need to change max features

outliers_predicted = outlier_detect.fit_predict(X)
now = time.time()
print(now- current, 'seconds')


105.32008814811707 seconds


In [324]:
outliers_predicted[outliers_predicted==-1].shape

(398,)

In [309]:
df_project['iForest_outlier'] = outliers_predicted

In [310]:
df_project.to_csv('anomaly_model_v4.csv',index=False)

In [318]:
temp = outlier_detect.decision_function(X)

In [313]:
outliers_predicted[1:3]

array([-1,  1])

In [169]:
'''
In practice,taking n_neighbors=20 appears to work well in general. When the proportion of outliers is high 
(i.e. greater than 10 %, as in the example below), n_neighbors should be greater (n_neighbors=35 in the example below). 
how isolated it is with respect to the surrounding neighborhood.
'''

current = time.time()
LOF = LocalOutlierFactor(n_neighbors=20,contamination = 0.001)
y_pred = LOF.fit_predict(X)
now = time.time()
print(now- current, 'seconds')

41.6827597618103 seconds


In [170]:
df_project['LOF_outlier'] = y_pred

In [82]:
df_project.head()

,project_type_name_eng,purchase_method_group_name_eng,province_eng,project_status_eng,district_eng,announce_date_eng,transaction_date_eng,project_money_cleaned,price_build_cleaned,sum_price_agree_cleaned,dept_name_eng,budget_year_cleaned,project_id,project_name,geom,dept_name_group,Topic_group,Iso_Forest_outlier,LOF_outlier
0,Hire for construction,Bidding,Nonthaburi,In progress,Bang Kruai,2 Apr 2013,9 March 2015,0,0,70624000000,Electricity Generating Authority of Thailand,2015,56045006726,ประกวดราคาจ้างซื้อและจ้างก่อสร้างโรงไฟฟ้าเพื่อ...,POINT(100.498178770987 13.8068549692447),Other,4,-1,1
1,Outsourcing / Contracting services,Bidding,Nonthaburi,In progress,Bang Kruai,21 Nov 2014,26 Aug 2015,30000000000,26910000000,22871000000,Electricity Generating Authority of Thailand,2015,57115200071,ประกวดราคาจ้างเหมาขุด-ขนดินและถ่าน ที่เหมืองแม...,POINT(100.498178770987 13.8068549692447),Other,14,-1,1
2,Outsourcing / Contracting services,E-bidding,Bangkok,In progress,Din,27 Aug 2014,13 Oct 2014,4455190000,4480010000,4436575000,Bangkok,2015,57085094977,ประกวดราคาจ้างเหมาเอกชนขนมูลฝอยจากศูนย์กำจัดมู...,POINT(100.565063509801 13.7780980259196),Other,14,1,1
3,buy,Special,Bangkok,In progress,Dusit,NaN,28 Jan 2015,2800000000,2799999972,2725935375,Army,2015,57085194402,ซื้อฮ.ท.ขนาดเบา แบบที่ 2 จำนวน 6 เครื่อง โดยวิ...,POINT(100.519873059321 13.7873113160748),Other,6,1,1
4,Hire for construction,E-bidding,Bangkok,In progress,Chatuchak,25 Mar 2014,9 Oct 2014,2599999769,2599999769,2588950000,Army,2015,57035180671,ประกวดราคาจ้างก่อสร้างอาคารเฉลิมพระเกียรติ สมเ...,POINT(100.584152190858 13.8453156541146),Other,3,1,1


In [171]:
df_project.to_csv('test_LOF.csv',index=False)

In [173]:
current = time.time()

LOF2 = LocalOutlierFactor(n_neighbors=20)
y_pred2 = LOF2.fit_predict(X)

now = time.time()
print(now- current, 'seconds')

44.07441425323486 seconds


In [305]:
type(OH_cols_train['x0_Hire consultants'])

pandas.core.series.Series